In [1]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import mayavi
import mayavi.mlab
import string
import subprocess
import functools
#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


In [2]:
def import_shit():
    import local_models.local_models
    import local_models.algorithms
    import local_models.utils
    import local_models.linear_projections
    import local_models.loggin
    import local_models.TLS_models
    import numpy as np
    import logging
    import string
    import ml_battery.log


    logger = logging.getLogger(__name__)

    #reload(local_models.local_models)
    #reload(lm)
    #reload(local_models.loggin)
    #reload(local_models.TLS_models)
    np.warnings.filterwarnings('ignore')
    return logger

In [3]:
FRESH=True

In [4]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [5]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [6]:
data = np.loadtxt("../data/bunny/bunny/data/bun.csv")

In [7]:
data.shape

(362272, 3)

In [8]:
(data.shape[0]*100)**(1/3)

330.8858963590516

In [9]:
np.mean(data, axis=0)

array([-0.02360042,  0.09758145,  0.01090169])

In [10]:
KERNEL=local_models.local_models.TriCubeKernel
RUN = 5

project_dir = "../data/bunny_scms_{}_{:03d}".format(kernel_names[KERNEL], RUN)

os.makedirs(project_dir, exist_ok=1)

In [11]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(2))
linear_models.fit(data)

In [12]:
avg_pt_dist = np.average(linear_models.index.query(data, k=2)[0][:,1])

In [13]:
avg_pt_dist

0.00026888393320762005

In [16]:
random_data_subset = data[np.random.randint(data.shape[0], size=50)]

In [20]:
queried = linear_models.index.query_radius(random_data_subset, r=avg_pt_dist*10)

In [21]:
list(map(lambda x: x.shape, queried))

[(165,),
 (207,),
 (144,),
 (112,),
 (142,),
 (188,),
 (203,),
 (156,),
 (221,),
 (187,),
 (138,),
 (177,),
 (167,),
 (178,),
 (193,),
 (106,),
 (124,),
 (170,),
 (205,),
 (184,),
 (218,),
 (149,),
 (150,),
 (157,),
 (138,),
 (212,),
 (145,),
 (214,),
 (142,),
 (202,),
 (216,),
 (147,),
 (181,),
 (157,),
 (219,),
 (175,),
 (204,),
 (183,),
 (154,),
 (110,),
 (93,),
 (93,),
 (138,),
 (155,),
 (125,),
 (180,),
 (159,),
 (218,),
 (88,),
 (195,)]

In [22]:
kernel = local_models.local_models.TriCubeKernel(bandwidth=avg_pt_dist*10)

In [23]:
linear_params_vecs, linear_params_mean = local_models.linear_projections.transformate_data(data, kernel, linear_models, r=kernel.support_radius())

In [28]:
linear_params_vecs.shape

(362272, 2, 3)

In [34]:
N = int(data.shape[0]/50)
sample_indices = np.random.choice(np.arange(data.shape[0]), size=N)

pth = os.path.join(project_dir, "single_convergence.png")
data_avg = np.average(data, axis=0)
data_std = np.std(data, axis=0)
figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
data_nodes = plt_data(figure, linear_params_mean[sample_indices], data_std)
normals = np.cross(*np.rollaxis(linear_params_vecs[sample_indices],1))
normal_vecs = plt_normals(figure, normals, linear_params_mean[sample_indices], data_std)
neg_normal_vecs = plt_normals(figure, -normals, linear_params_mean[sample_indices], data_std)
normalize_view(figure, data_avg, data_std)
mayavi.mlab.savefig(pth, magnification=2)
mayavi.mlab.close(figure)

In [14]:
#kernel_kwargses = [{"bandwidth": x} for x in np.linspace(avg_pt_dist, 100*avg_pt_dist, 20)]
kernel_kwargses = [{"bandwidth": "knn", "k": x} for x in range(4,40,2)]

In [15]:
pth = os.path.join(project_dir, "filler")
os.makedirs(pth, exist_ok=1)

In [16]:
def mean_center(data, weights=None):
    return data - np.average(data, axis=0,weights=weights)

In [26]:
def load_converged_data(pth):
    convergededs = []
    for dat in sorted(os.listdir(pth)):
        convergededs.append(np.loadtxt(os.path.join(pth, dat)))
    return np.concatenate(convergededs, axis=0)

def plt_grid(fig, grid, data_avg, data_std):
    nodes = mayavi.mlab.points3d(grid[:,0], grid[:,1], grid[:,2], 
                                 scale_mode='scalar', scale_factor=1,
                                 colormap='gist_earth', figure=fig)
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((grid.shape[0],3))*(np.average(data_std)/60)
    nodes.mlab_source.dataset.point_data.scalars = (grid[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    return nodes

def plt_data(fig, data, data_std):
    nodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], 
                                 scale_mode='scalar', scale_factor=1,
                                 colormap='Greens', figure=fig)
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((data.shape[0],3))*(np.average(data_std)/60)
    nodes.mlab_source.dataset.point_data.scalars = np.ones((data.shape[0]))
    return nodes

def get_normals(kernel, linear_models, data):
    if hasattr(kernel.bandwidth, "__call__"):
        linear_params_vecs, linear_params_mean = local_models.linear_projections.transformate_data(data, kernel, linear_models, k=kernel.k)
    else:
        linear_params_vecs, linear_params_mean = local_models.linear_projections.transformate_data(data, kernel, linear_models, r=kernel.support_radius())
    return linear_params_vecs

def align_normals(data, normals, k=10, iterations=100):
    balltree = sklearn.neighbors.BallTree(data)
    pairwise_nearest_indices = balltree.query(data,k=k,sort_results=True,return_distance=False)
    for iteration in range(iterations):
        alignments = []
        for index in range(1,pairwise_nearest_indices.shape[1]):
            alignment = np.einsum("ij,ij->i",normals,normals[pairwise_nearest_indices[:,index]])
            alignments.append(alignment)
        alignment = np.average(alignments, axis=0)
        wrong_alignment = np.sign(alignment)
        normals = normals*wrong_alignment.reshape(-1,1)
    return normals

def align_edge_normals(data, normals, edge_range=0.1):
    data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
    graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, -edge_range)
    mins = data < graph_bounds[:1]
    maxes = data > graph_bounds[1:]
    mins_alignment = np.sign(np.einsum("ij,ij->i",mins,-1*normals))
    maxes_alignment = np.sign(np.einsum("ij,ij->i",maxes,normals))
    mins_alignment += np.logical_not(mins_alignment) # turn 0s into 1s (so they don't change)
    maxes_alignment += np.logical_not(maxes_alignment)    
    return normals*mins_alignment.reshape(-1,1)*maxes_alignment.reshape(-1,1)

def plt_normals(fig, normals, data, data_std):
    nodes = mayavi.mlab.quiver3d(data[:,0], data[:,1], data[:,2],
                                 normals[:,0], normals[:,1], normals[:,2],
                                 scale_mode='scalar', scale_factor=np.average(data_std)/5,
                                 colormap='Purples', figure=fig, line_width=1.0)
    return nodes

def normalize_view(fig, data_avg, data_std):
    mayavi.mlab.view(azimuth=0, elevation=0, distance=15*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
    
def plt_and_save(data, grid, normals, pth):
    data_avg = np.average(data, axis=0)
    data_std = np.std(data, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    data_nodes = plt_data(figure, data, data_std)
    converged_nodes = plt_grid(figure, grid, data_avg, data_std)
    normal_vecs = plt_normals(figure, normals, grid, data_std)
    neg_normal_vecs = plt_normals(figure, -normals, grid, data_std)
    normalize_view(figure, data_avg, data_std)
    mayavi.mlab.savefig(pth, magnification=2)
    mayavi.mlab.close(figure)

def serialize_plt(pth):
    import zlib
    with open(pth, 'rb') as f:
        dat = f.read()
    return zlib.compress(dat)
   
def deserialize_plt(dat, pth):
    import zlib
    with open(pth, 'wb') as f:
        f.write(zlib.decompress(dat))
    return pth

def distributed_plt_and_save(data, grid, bandwidth):
    import numpy as np
    import mayavi
    import mayavi.mlab
    import string
    import os
    #on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
    mayavi.mlab.options.offscreen = True
    
    unique_id = "".join(np.random.choice(list(string.ascii_lowercase), replace=True, size=20))
    pth = "/ramfs/{}.png".format(unique_id)
    try:
        plt_and_save(data, grid, bandwidth, pth)
        result = serialize_plt(pth)
    except:
        os.remove(pth)

In [18]:
for i,kernel_kwargs in enumerate(kernel_kwargses):
    kernel = KERNEL(**kernel_kwargs)
    data_pth = os.path.join(pth, str(kernel))
    title = str(kernel)
    img_pth = os.path.join(pth, "{}.png".format(title))
    grid = load_converged_data(data_pth)
    normals = np.cross(*np.rollaxis(get_normals(kernel, linear_models, grid),1))
    print(i)
    plt_and_save(data, grid, normals, img_pth)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [19]:
top_pt = data[18765]
def normalize_view(fig, data_avg, data_std):
    mayavi.mlab.view(azimuth=0, elevation=0, distance=5*np.average(data_avg), focalpoint=(top_pt[0], top_pt[1], top_pt[2]))

In [20]:
import sys
for var, obj in sorted(locals().items(), key=lambda x:-sys.getsizeof(x[1])):
    if sys.getsizeof(obj) > 5000:
        print(var, sys.getsizeof(obj))
for var, obj in sorted(globals().items(), key=lambda x:-sys.getsizeof(x[1])):
    if sys.getsizeof(obj) > 5000:
        print(var, sys.getsizeof(obj))
import gc
gc.collect()
mayavi.mlab.close(all=True)

grid 168112
normals 168112
grid 168112
normals 168112


In [21]:
for i,kernel_kwargs in enumerate(kernel_kwargses):
    kernel = KERNEL(**kernel_kwargs)
    data_pth = os.path.join(pth, str(kernel))
    title = str(kernel)
    img_pth = os.path.join(pth, "{}_small.png".format(title))
    grid = load_converged_data(data_pth)
    normals = np.cross(*np.rollaxis(get_normals(kernel, linear_models, grid),1))
    normals = align_edge_normals(grid, normals)
    normals = align_normals(grid, normals,iterations=100,k=50)
    normals = align_edge_normals(grid, normals)
    normals = align_normals(grid, normals,iterations=100,k=30)
    normals = align_edge_normals(grid, normals)
    normals = align_normals(grid, normals,iterations=100,k=10)
    plt_and_save(data, grid, normals, img_pth)

In [ ]:
import sys
for var, obj in sorted(locals().items(), key=lambda x:-sys.getsizeof(x[1])):
    if sys.getsizeof(obj) > 5000:
        print(var, sys.getsizeof(obj))
for var, obj in sorted(globals().items(), key=lambda x:-sys.getsizeof(x[1])):
    if sys.getsizeof(obj) > 5000:
        print(var, sys.getsizeof(obj))
import gc
gc.collect()
mayavi.mlab.close(all=True)


In [43]:
np.max(data, axis=0)

array([0.06137 , 0.18794 , 0.059117])

In [44]:
np.max(grid, axis=0)

array([-0.02331136,  0.18223789,  0.05456296])

In [45]:
data.shape

(362272, 3)

In [46]:
grid.shape

(25000, 3)

In [24]:
grid[0]

array([-0.15041934,  0.01747047, -0.09899874])

In [ ]:
bandwidths = [BANDWIDTH]
converged = np.concatenate(convergededs)
for i, bandwidth in enumerate(bandwidths):
    data_avg = np.average(converged, axis=0)
    data_std = np.std(converged, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    #mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
    nodes = mayavi.mlab.triangular_mesh(converged[:,0], converged[:,1], converged[:,2], 
                                 scale_mode='scalar', colormap='gist_earth')
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((converged.shape[0],3))*(np.average(data_std)/400)
    nodes.mlab_source.dataset.point_data.scalars = (converged[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    #bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
    #bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


    #mayavi.mlab.axes()

    #mayavi.mlab.view(views[0][1],views[0][0])
    mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
    mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0])
    title = "converged_data_b{:08.05f}".format(bandwidth)
    mayavi.mlab.savefig(os.path.join(pth, "{}.png".format(title)))
    mayavi.mlab.close(figure)

In [ ]:
mayavi.mlab.close(all=True)

In [ ]:
range_pct = 0.2
grid_steps = 20
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid_limits = tuple(map(lambda i: slice(graph_bounds[0,i], graph_bounds[1,i], grid_steps*1j), range(graph_bounds.shape[1])))
grid = np.mgrid[grid_limits]
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [ ]:
for converged2 in local_models.algorithms.local_tls_shift_till_convergence(linear_models, grid, tol=1e-10,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [ ]:
converged2

In [ ]:
cloud = pcl.PointCloud()
cloud.from_array(converged2.astype('float32'))
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [ ]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
nodes = mayavi.mlab.points3d(converged2[:,0], converged2[:,1], converged2[:,2], scale_mode='scalar', color=(1,0,0))
nodes.mlab_source.dataset.point_data.scalars = np.ones(converged2.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(converged2, axis=1)
mayavi.mlab.view(azimuth=0, elevation=30, distance=1, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "converged_g000{:05d}".format(grid_steps)
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()